In [2]:
import vcf

In [3]:
vcf_path = "/home/djeffrie/Data/RADseq/Riberica/Stacks_trimmed/IDd/FAMILY/batch_1.vcf"
popmap = open("/home/djeffrie/Data/RADseq/Riberica/Stacks_trimmed/IDd/FAMILY/popmap.txt", 'r').readlines()

alteredvcfpath = "%s%s" % (vcf_path, ".altered")

oldvcf = open(vcf_path, 'r').readlines()
alteredvcf = open(alteredvcfpath, 'w')        

for line in oldvcf:
    
    if "Allele Depth" not in line:            
        alteredvcf.write(line)
    elif "Allele Depth" in line:            
        line = '##FORMAT=<ID=AD,Number=.,Type=Integer,Description="Allele Depth">\n'
        alteredvcf.write(line)
        
alteredvcf.close()    
altered_vcf = open(alteredvcfpath, 'r')    
myvcf = vcf.Reader(altered_vcf)


In [4]:
altered_vcf = open(alteredvcfpath, 'r')    
myvcf = vcf.Reader(altered_vcf)

per_sample_dict = {}

loc_order_list = []
sample_order_list = []


for record in myvcf:
    #print record
    loc_id = "%s_%s" % (record.ID, record.POS)
    
    loc_order_list.append(loc_id)
    
    for sample in record:
        if sample.sample not in per_sample_dict:
            per_sample_dict[sample.sample] = {}
            sample_order_list.append(sample.sample)
        
        if sample['GT'] == None:
            new_gt = "0 0"
        elif sample['GT'] == "0/0":
            new_gt = "1 1"
        elif sample['GT'] == "0/1":
            new_gt = "1 2"
        elif sample['GT'] == "1/0":
            new_gt = "2 1"
        elif sample['GT'] == "1/1":
            new_gt = "2 2"
        
        per_sample_dict[sample.sample][loc_id] = new_gt
            
sample_loc_lists = {}     
sample_lines = {}

for sample in sample_order_list:
    sample_loc_lists[sample] = []
    
    
    for loc in loc_order_list:
        sample_loc_lists[sample].append(per_sample_dict[sample][loc])

    sample_lines[sample] = "\t".join(sample_loc_lists[sample])

In [5]:
samples = []

for line in popmap:
    samples.append(line.split()[0])
print samples

['P1', 'P2', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'T17', 'T18', 'T19', 'T1', 'T20', 'T21', 'T22', 'T23', 'T24', 'T25', 'T26', 'T27', 'T28', 'T29', 'T2', 'T30', 'T31', 'T32', 'T33', 'T34', 'T35', 'T36', 'T37', 'T38', 'T39', 'T3', 'T40', 'T41', 'T44', 'T45', 'T46', 'T47', 'T48', 'T49', 'T4', 'T50', 'T51', 'T52', 'T53', 'T54', 'T55', 'T56', 'T57', 'T58', 'T59', 'T5', 'T60', 'T61', 'T62', 'T63', 'T64', 'T6', 'T7', 'T8', 'T9']


In [6]:
outfile = open("/home/djeffrie/Data/RADseq/Riberica/Stacks_trimmed/IDd/FAMILY/LEPMAP/Riberica_lepmap_input.data", 'w')

for sample in samples:
    outfile.write("%s\t%s\n" % (sample, sample_lines[sample]))
                  
outfile.close()
